In [2]:
from src.utils import collapse_isoforms_to_proteoforms, read_gtf
import polars as pl

# Add base_isoform to protein_classification

In [3]:
tx_classification = pl.read_parquet("nextflow_results/V47/final_classification.parquet")

orfanage_gtf = read_gtf("nextflow_results/V47/orfanage/orfanage.gtf", attributes=["gene_id", "transcript_id"])
protein_classification = pl.read_csv("nextflow_results/V47/orfanage/SFARI.protein_classification.tsv", separator="\t")
protein_classification = protein_classification\
    .join(
        collapse_isoforms_to_proteoforms(orfanage_gtf).rename({"isoform": "pb"}),
        on = "pb",
        how = "left"
    )

/gpfs/fs0/scratch/s/shreejoy/nxu/SFARI/src/utils.py:7013: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  .with_columns(


In [17]:
protein_classification.write_csv("export/SFARI.protein_classification.tsv", separator="\t")

In [8]:
tx_classification\
    .filter(
        pl.col("isoform").str.starts_with("PB.44.")
    )

isoform,chrom,strand,length,exons,structural_category,associated_gene,associated_transcript,ref_length,ref_exons,diff_to_TSS,diff_to_TTS,diff_to_gene_TSS,diff_to_gene_TTS,subcategory,RTS_stage,all_canonical,min_sample_cov,min_cov,min_cov_pos,sd_cov,FL,n_indels,n_indels_junc,bite,iso_exp,gene_exp,ratio_exp,FSM_class,coding,ORF_length,CDS_length,CDS_start,CDS_end,CDS_genomic_start,CDS_genomic_end,predicted_NMD,perc_A_downstream_TTS,seq_A_downstream_TTS,dist_to_CAGE_peak,within_CAGE_peak,dist_to_polyA_site,within_polyA_site,polyA_motif,polyA_dist,polyA_motif_found,ORF_seq,ratio_TSS,fl_assoc,cell_barcodes,containing_novel_spl
str,str,str,i32,i32,str,str,str,i32,i32,i32,i32,i32,i32,str,str,str,str,str,str,str,str,str,str,bool,str,str,str,str,str,str,str,str,str,str,str,str,f64,str,str,str,str,str,str,str,str,str,str,str,str,bool
"""PB.44.2""","""chr1""","""-""",2173,7,"""full-splice_match""","""SDF4""","""ENST00000360001.12""",1933,7,-224,-16,-194,16,"""alternative_5end""","""FALSE""","""canonical""",null,null,null,null,null,null,null,false,null,null,null,"""C""","""non_coding""",null,null,null,null,null,null,null,10.0,"""AGCTTCATTCTTTGGGCGTG""",null,null,null,null,null,null,null,null,null,null,null,false
"""PB.44.3""","""chr1""","""-""",2094,8,"""novel_not_in_catalog""","""SDF4""","""novel""",1933,7,null,null,-34,23,"""at_least_one_novel_splicesite""","""FALSE""","""canonical""",null,null,null,null,null,null,null,true,null,null,null,"""C""","""non_coding""",null,null,null,null,null,null,null,30.0,"""TTCTTTGGGCGTGAAACAAA""",null,null,null,null,null,null,null,null,null,null,null,true
"""PB.44.4""","""chr1""","""-""",1990,7,"""novel_in_catalog""","""SDF4""","""novel""",2095,7,null,null,-20,-6,"""combination_of_known_junctions""","""FALSE""","""canonical""",null,null,null,null,null,null,null,false,null,null,null,"""C""","""non_coding""",null,null,null,null,null,null,null,25.0,"""TTAAATATAGTTGCCTTTTT""",null,null,null,null,null,null,null,null,null,null,null,false
"""PB.44.6""","""chr1""","""-""",1849,6,"""novel_in_catalog""","""SDF4""","""novel""",1933,7,null,null,-24,21,"""combination_of_known_splicesit…","""FALSE""","""canonical""",null,null,null,null,null,null,null,false,null,null,null,"""C""","""non_coding""",null,null,null,null,null,null,null,25.0,"""CATTCTTTGGGCGTGAAACA""",null,null,null,null,null,null,null,null,null,null,null,false
"""PB.44.7""","""chr1""","""-""",2229,7,"""full-splice_match""","""SDF4""","""ENST00000263741.12""",2079,7,-157,7,-157,-4,"""alternative_5end""","""FALSE""","""canonical""",null,null,null,null,null,null,null,false,null,null,null,"""C""","""non_coding""",null,null,null,null,null,null,null,25.0,"""AAATATAGTTGCCTTTTTTG""",null,null,null,null,null,null,null,null,null,null,null,false
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""PB.44.2198""","""chr1""","""-""",1040,6,"""full-splice_match""","""CCNL2""","""ENST00000408918.8""",1186,6,-11,157,0,-157,"""alternative_3end""","""FALSE""","""canonical""",null,null,null,null,null,null,null,false,null,null,null,"""C""","""non_coding""",null,null,null,null,null,null,null,20.0,"""AGTGTGGGGTTATGTCATGA""",null,null,null,null,null,null,null,null,null,null,null,false
"""PB.44.2253""","""chr1""","""-""",1242,4,"""full-splice_match""","""MRPL20""","""ENST00000344843.12""",700,4,-541,-1,-541,1,"""alternative_5end""","""FALSE""","""canonical""",null,null,null,null,null,null,null,false,null,null,null,"""C""","""non_coding""",null,null,null,null,null,null,null,15.0,"""TTTTTAGAGTCATTTTCTGG""",null,null,null,null,null,null,null,null,null,null,null,false
"""PB.44.2254""","""chr1""","""-""",1391,4,"""novel_in_catalog""","""MRPL20""","""novel""",700,4,null,null,-9,-1,"""intron_retention""","""FALSE""","""canonical""",null,null,null,null,null,null,null,false,null,null,null,"""C""","""non_coding""",null,null,null,null,null,null,null,20.0,"""AGTTTTTAGAGTCATTTTCT""",null,null,null,null,null,null,null,null,null,null,null,false


# Make reduced ORFanage GTF for viewing in UCSC browser

In [ ]:
orfanage_gtf\
    .with_columns(
        attributes = pl.lit('gene_id "')+pl.col("gene_id")+pl.lit('"; transcript_id "')+pl.col("transcript_id")+pl.lit('";')
    )\
    .drop("gene_id", "transcript_id")\
    .write_csv(
        "export/orfanage_mini.gtf",
        separator = "\t",
        quote_style="never",
        include_header=False
    )

: 